In [3]:
!pip3 install -q transformers
!pip3 install -q simpletransformers

In [4]:
import pandas as pd

df = pd.read_csv('/content/Dynamically Generated Hate Dataset v0.2.2.csv', on_bad_lines='skip')


In [5]:
df.head()

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
4,5,acl10000,5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,1,NaN


In [6]:
df=df.drop(['Unnamed: 0'],axis=1)

In [7]:
df = df.replace({'label': {'hate': 1, 'nothate': 0}})
df = df.rename(columns={'label': 'labels'})
df.head()

<ipython-input-7-770224abb1c9>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'label': {'hate': 1, 'nothate': 0}})


,acl.id,X1,text,labels,type,target,level,split,round.base,annotator,round,acl.id.matched
0,acl1,1,dalits are lowlives,1,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,acl10,2,gay people are a burden to our society,1,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,acl100,3,Arabs are not welcome,1,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,acl1000,4,I'm not saying we should actually eliminate he...,1,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
4,acl10000,5,bananas are for black people,1,notgiven,notgiven,original,test,1,QiOKkCi7F8,1,NaN


In [8]:
train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']
data = [[train_df.labels.value_counts()[0], test_df.labels.value_counts()[0]],
        [train_df.labels.value_counts()[1], test_df.labels.value_counts()[1]]]
# Prints out the dataset sizes of train test and validate as per the table.
pd.DataFrame(data, columns=['Train', 'Test'])

,Train,Test
0,15202,1853
1,17804,2271


In [9]:

train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'train_batch_size': 128,
    'fp16': True,
    'output_dir': '/outputs/',
}

In [10]:

from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the XLNet base cased pre-trained model.
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=train_args)

# Train the model, there is no development or validation set for this dataset
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Running Epoch 1 of 1:   0%|          | 0/299 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/4124 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/47 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [11]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.f1_score)

  0%|          | 0/4124 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/47 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [12]:

samples = [test_df[test_df['labels'] == 0].sample(1).iloc[0]['text']] # get a random sample from the test set which is nothate
predictions, _ = model.predict(samples)
label_dict = {0: 'nothate', 1: 'hate'}
for idx, sample in enumerate(samples):
  print('{} - {}: {}'.format(idx, label_dict[predictions[idx]], sample))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 - hate: I will prove you are wrong by believing that Indians are useless currymunchers


/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [13]:
predictions, _ = model.predict(test_df['text'].tolist())
df = pd.DataFrame(predictions)
df.to_csv('results.txt', index=False, header=False) # saves the prediction results to a file in the colab environment



  0%|          | 0/4124 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

In [14]:
def predict_hate_speech(text):
    # Predict the sentiment of the input text
    prediction, _ = model.predict([text])  # Pass the text as a list for prediction
    # Return "Hate Speech" if the prediction is 1, otherwise "Not Hate Speech"
    return 'Hate Speech' if prediction[0] == 1 else 'Not Hate Speech'

In [16]:
def get_user_input():
    user_input = input("Enter text to check for hate speech: ")
    result = predict_hate_speech(user_input)
    print(f'Result: {result}')

get_user_input()

Enter text to check for hate speech: these people are lowlives


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Result: Hate Speech


Enter text to check for hate speech: I love india
100% 1/1 [00:00<00:00,  4.55it/s]

100% 1/1 [00:00<00:00, 23.10it/s]

Result: Not Hate Speech